# Argos Query Suggestions

pip install fake_useragent

In [3]:
import requests
import json
from fake_useragent import UserAgent

In [4]:
ua = UserAgent()
headers = {"user-agent": ua.chrome}

In [5]:
import pandas as pd
import numpy as np

### Loading list of toys

In [1]:
with open('all_items.txt') as f:
    contents = f.read().splitlines()
#contents

## Loading pre-documented gender stereotyped toys

In [8]:
stereo_toys = pd.read_csv('~/Desktop/485/Data/predoc_stereotyped_items.csv', delimiter =',')
stereo_toys

,BOY,GIRL,NEUTRAL
0,vehicle toys,doll,toy animals
1,sport,domestic toys,books
2,military toys,educational art,educational teaching
3,race cars,clothes,musical games
4,outer space toys,dollhouses,games
...,...,...,...
67,toy rocket,barbie furniture set,NaN
68,soccer ball,NaN,NaN
69,blue ipad,pink ipad,NaN
70,toy robots,NaN,NaN


stereo_toys['BOY'] = stereo_toys['BOY'].apply(lambda x: x.lower() if isinstance(x, str) else x)
stereo_toys['GIRL'] = stereo_toys['GIRL'].apply(lambda x: x.lower() if isinstance(x, str) else x)
stereo_toys['NEUTRAL'] = stereo_toys['NEUTRAL'].apply(lambda x: x.lower() if isinstance(x, str) else x)
stereo_toys

stereo_toys.to_csv('../data/stereotyped_items.csv', index=False)

### Some stat on the dataset

In [9]:
boy_toys = stereo_toys['BOY'].unique().tolist()
len(boy_toys)

71

In [10]:
girl_toys = stereo_toys['GIRL'].unique().tolist()
len(girl_toys)

64

In [11]:
neutral_toys = stereo_toys['NEUTRAL'].unique().tolist()
len(neutral_toys)

33

In [12]:
all_items = boy_toys + girl_toys + neutral_toys
len(all_items)

168

In [13]:
all_items = [x for x in all_items if isinstance(x, str)]

## Preparing the data for query

textfile = open("all_items.txt", "w")
for element in all_items:
    textfile.write(element + "\n")
textfile.close()

In [12]:
#contents

### Adding "for"

In [14]:
new_content = [x.strip() for x in all_items]
new_content = [x.replace(' for','') for x in new_content]
new_content = [x.lower() for x in new_content]
new_content

['vehicle toys',
 'sport',
 'military toys',
 'race cars',
 'outer space toys',
 'depots',
 'machines',
 'doll-humanoid',
 'action figures',
 'gi joe action figure',
 'spiderman',
 'guitar',
 'castle tent',
 'microscope',
 'monster trucks',
 'bug collection set',
 'barn',
 'grill',
 'dinosaur toy',
 'volcano creator',
 'pokemon cards',
 'space station toys',
 'basketball hoop',
 'gears',
 'train set',
 'lincoln logs',
 'police station toy',
 'airport toy',
 'police officer gear',
 'dragonballz',
 'superhero costumes',
 'helicopter',
 'racetrack',
 'remote control car',
 'football',
 'hockey stick',
 'sword toy',
 'miniature guns',
 'truck set',
 'remote control helicopter',
 'sully costume',
 'woody valentine',
 'spiderman costume',
 'batman costume',
 'ninja costume',
 'wwe action figure',
 'action figure',
 'toy gun',
 'construction games',
 'war toys',
 'toy vehicles',
 'educational material',
 'nerf guns',
 'construction toys',
 'car toys',
 'video games',
 'building blocks',
 'din

In [15]:
search_terms = [x+' for' for x in new_content]
search_terms

['vehicle toys for',
 'sport for',
 'military toys for',
 'race cars for',
 'outer space toys for',
 'depots for',
 'machines for',
 'doll-humanoid for',
 'action figures for',
 'gi joe action figure for',
 'spiderman for',
 'guitar for',
 'castle tent for',
 'microscope for',
 'monster trucks for',
 'bug collection set for',
 'barn for',
 'grill for',
 'dinosaur toy for',
 'volcano creator for',
 'pokemon cards for',
 'space station toys for',
 'basketball hoop for',
 'gears for',
 'train set for',
 'lincoln logs for',
 'police station toy for',
 'airport toy for',
 'police officer gear for',
 'dragonballz for',
 'superhero costumes for',
 'helicopter for',
 'racetrack for',
 'remote control car for',
 'football for',
 'hockey stick for',
 'sword toy for',
 'miniature guns for',
 'truck set for',
 'remote control helicopter for',
 'sully costume for',
 'woody valentine for',
 'spiderman costume for',
 'batman costume for',
 'ninja costume for',
 'wwe action figure for',
 'action figur

search_books = [x+' for' for x in books]
search_books

## Search Engines and E-Com Sites

### Argos

In [16]:
def argos_scrape(query):
    url = f'https://www.argos.co.uk/suggest?term={query}%7Bq%7D%20f&fuzziness=true&size=5&includeFaq=true'
    response = requests.get(url, headers=headers, verify=False).json()
    return response

### All together

In [17]:
def argos_auto(item):
    res = argos_scrape(item)
    suggestions = res['suggestions']
    results = []
    for s in suggestions:
        results.append(s['value'])
    return results

## Run queries

In [18]:
import pandas as pd
columns = ['platform', 'query', 'suggestions']
df = pd.DataFrame(columns = columns)
df

,platform,query,suggestions


In [19]:
def search(platform):
    if platform == 'Argos':
        return argos_auto(item)

In [20]:
import time
import warnings
warnings.filterwarnings('ignore')
platforms = ['Argos']
data = []
for item in contents:
    for platform in platforms:
        result = search(platform) 
        values = [platform, item, result]
        zipped = zip(columns, values)
        a_dictionary = dict(zipped)
        #print(a_dictionary)
        time.sleep(1.5)
        data.append(a_dictionary)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [39]:
data

[{'platform': 'Amazon_UK',
  'query': 'vehicle toys',
  'suggestions': ['vehicle toys',
   'vehicle toys for boys 3-5 years old',
   'vehicle toys for 3 year old boys',
   'vehicle toys for toddlers 1-3',
   'vehicle toys for boys 5-8',
   'vehicle toys for 2 year old boys',
   'vehicle toys for 4 year old boys',
   'vehicle toys for toddlers 2',
   'vehicle toys for infants',
   'vehicle toys for 5 year old boys']},
 {'platform': 'Amazon_UK',
  'query': 'sport',
  'suggestions': ['sports bras for women',
   'sports bra',
   'sports gifts for boys 8-12',
   'sports',
   'sports water bottle',
   'sports bras for women high support large bust',
   'sports cards',
   'sports card binder',
   'sports gifts for men',
   'sports bag']},
 {'platform': 'Amazon_UK',
  'query': 'military toys',
  'suggestions': ['military toys',
   'military toys for boys age 8-12',
   'military toys for boys age 4-7',
   'military toys for boys',
   'military toys for boys age 6-12',
   'military toys for kids

In [40]:
df = df.append(data, True)
df

,platform,query,suggestions
0,Amazon,vehicle toys,"[vehicle toys, vehicle toys for boys 3-5 years..."
1,Amazon,sport,"[sports bras for women, sports bra, sports gif..."
2,Amazon,military toys,"[military toys, military toys for boys age 8-1..."
3,Amazon,race cars,"[race cars, race cars for kids ages 3-5, race ..."
4,Amazon,outer space toys,"[outer space toys, outer space toys for kids 5..."
...,...,...,...
261,Amazon_UK,playhouses,"[playhouses, playhouses for kids outdoor, play..."
262,Amazon_UK,princess sword,"[princess sword, princess sword and shield, pr..."
263,Amazon_UK,doctors kit,"[doctors kit, doctors kit for 3 year old, doct..."
264,Amazon_UK,skin care kit,"[skin care kit, skin care kit for teen girls, ..."


In [41]:
df.to_csv('az_uk_query_suggestions.csv', index = False)